In [1]:
class Solution:
    def minimalSteps(self, maze):
        s_i, s_j = -1, -1
        t_i, t_j = -1, -1
        m_pos = []      # 机关
        o_pos = []      # 石堆

        m, n  = len(maze), len(maze[0])
        for i in range(m):
            for j in range(n):
                ch = maze[i][j]
                if ch == 'M':
                    m_pos.append((i, j))
                elif ch == 'O':
                    o_pos.append((i, j))
                elif ch == 'S':
                    s_i, s_j = i, j
                elif ch == 'T':
                    t_i, t_j = i, j


        s2o_dist = self.min_dist_list(maze, s_i, s_j, o_pos) # 起点到每个石堆的最短距离
        t2m_dist= self.min_dist_list(maze, t_i, t_j, m_pos) # 终点到每一个机关的最短距离

        m2o_dist = [] # 每一个机关到每一个石堆的最短距离
        for m_idx, (m_i, m_j) in enumerate(m_pos):
            tmp_dist_list = self.min_dist_list(maze, m_i, m_j, o_pos)
            m2o_dist.append(tmp_dist_list)

        s2t_dist = self.min_dist_list(maze, s_i, s_j, [(t_i, t_j)])[0] # 起点到终点的最短距离
        if len(m_pos) == 0:
            return s2t_dist

        # 先提前计算好两个机关之间通过一个石堆中转的最小开销, 后面DP要用
        m2m_dist = [[0 for _ in range(len(m_pos))] for _ in range(len(m_pos))]
        for i in range(len(m_pos)):
            for j in range(i+1, len(m_pos)):
                min_sum = 0x7fffffff
                for o_idx in range(len(o_pos)):
                    if m2o_dist[i][o_idx] != -1 and m2o_dist[j][o_idx] != -1:
                        min_sum = min(min_sum, m2o_dist[i][o_idx] + m2o_dist[j][o_idx])
                if min_sum == 0x7fffffff:
                    min_sum = -1

                m2m_dist[i][j] = min_sum
                m2m_dist[j][i] = min_sum

        # 提前计算起点到下一个机关之间通过一个石碓中转最小开销，后面DP使用
        s2m_dist = [0 for _ in range(len(m_pos))]
        for m_idx in range(len(m_pos)):
            min_sum = 0x7fffffff
            for o_idx in range(len(o_pos)):
                if s2o_dist[o_idx] != -1 and m2o_dist[m_idx][o_idx] != -1:
                    min_sum = min(min_sum, s2o_dist[o_idx] + m2o_dist[m_idx][o_idx])
            s2m_dist[m_idx] = -1 if min_sum == 0x7fffffff else min_sum

        '''
        后面的问题是一个DP求最佳策略问题
        假设机关总共有x个
        求一个距离和最小的序列
        S O1 M1 O2 M2 ..... Ox Mx T
        其中S, T表示起点和终点位置，不可变动
        Oi Mi表示中间经过的石堆位置和机关位置, 可以从所有的可能的O和M中选择，但是M的数值不能在序列里面重复
        M总共只有16个，所以可以用位运算压缩状态的表示, 数位为1表示对应下标的M已经选择过了，反之表示没有选择过
        '''

        max_stat = (1 << (len(m_pos))) - 1
        # dp[stat][m_idx] 表示机关的选中状态为stat, 且最后一次选择m_idx下标的机关情况下的最小路径和
        dp = [[-1 for _ in range(len(m_pos))] for _ in range(max_stat+1)]
        for stat in range(1, max_stat+1):

            if stat & (stat - 1) == 0:
                # 只有一个位置被选中
                m_idx = 0
                while (1 << m_idx) & stat == 0:
                    m_idx += 1
                dp[stat][m_idx] = s2m_dist[m_idx]

            else:
                for m_idx in range(len(m_pos)):
                    if (1 << m_idx) & stat == 0:
                        continue

                    prev_stat = stat & (~((1 << m_idx)))
                    ans = 0x7fffffff
                    for prev_m_idx in range(len(m_pos)):
                        if ((1 << prev_m_idx) & prev_stat) and m2m_dist[prev_m_idx][m_idx] != -1 and dp[prev_stat][prev_m_idx] != -1:
                            ans = min(ans, m2m_dist[prev_m_idx][m_idx] + dp[prev_stat][prev_m_idx])
                    if ans == 0x7fffffff:
                        ans = -1
                    dp[stat][m_idx] = ans

        ans = 0x7fffffff
        for m_idx in range(len(m_pos)):
            if dp[max_stat][m_idx] != -1 and t2m_dist[m_idx] != -1:
                ans = min(ans, dp[max_stat][m_idx] + t2m_dist[m_idx])
        return -1 if ans == 0x7fffffff else ans


    # 求开始位置到结束位置列表中所有点的最短距离
    def min_dist_list(self, maze, start_i, start_j, end_list):
        pos2idx = {}
        for idx, (end_i, end_j) in enumerate(end_list):
            pos2idx[(end_i, end_j)] = idx


        m, n = len(maze), len(maze[0])
        que = deque()
        que.append((0, start_i, start_j))
        best_stat = {(start_i, start_j): 0}
        ans = [-1 for _ in range(len(end_list))]

        ans_cnt = 0
        while len(que) > 0:
            cost, i, j = que.popleft()
            # print(i, j, cost)

            if (i, j) in pos2idx:
                ans[pos2idx[(i, j)]] = cost
                ans_cnt += 1
                if ans_cnt == len(end_list):
                    break

            for ii, jj in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]:
                if ii >= 0 and ii < m and jj >= 0 and jj < n and maze[ii][jj] != '#':
                    # print(ii, jj)
                    if (ii, jj) not in best_stat or cost + 1 < best_stat[(ii, jj)]:
                        que.append((cost + 1, ii, jj))
                        best_stat[(ii, jj)] = cost + 1
        return ans